In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import hot
import pandas as pd
import typing
import os 
import random
import pickle
import torch
from typing import Union, List
from torch.utils.data import Dataset, DataLoader
from MyDataSet import MultiMaskTimeSeriesDataset

project_path = r'D:\WorkPath\Models\ImputeFormer\Data'

free_flow_file=project_path+r'\raw_data\Seattle\Loop_Seattle_2015_reachability_free_flow_5min.npy'
data_file=project_path+r'\raw_data\Seattle\speed_matrix_2015'

raw_data=pd.read_pickle(data_file)
raw_data.index=pd.to_datetime(raw_data.index)

node_files=project_path+r'\raw_data\Seattle\nodes_loop_mp_list.csv'
node=pd.read_csv(node_files,header=0,index_col=0)
node['direction']=node['milepost'].apply(lambda x: x[0])
node['route']=node['milepost'].apply(lambda x: x[1:4])
node['mile_post']=node['milepost'].apply(lambda x: float(x[6:]))

node.groupby(['direction','route']).count()

In [17]:
raw_data.index=pd.to_datetime(raw_data.index)
raw_data['day']=raw_data.index.time
raw_data['day']=raw_data['day'].apply(lambda x: (x.hour*3600+x.minute*60+x.second)*10**9)

In [18]:
def time_unit_to_nanoseconds(time_unit: str):
    # check_time_unit(time_unit)
    if time_unit == 'year':
        return 365.2425 * 24 * 60 * 60 * 10**9
    elif time_unit == 'week':
        time_unit = 'W'
    return pd.Timedelta('1' + time_unit).value

def datetime_encoded(dataset : pd.DataFrame, units: Union[str, List]) -> pd.DataFrame:
    r"""Transform dataset's temporal index into covariates using sinusoidal
    transformations. Each temporal unit is used as period to compute the
    operations, obtaining two feature (:math:`\sin` and :math:`\cos`) for
    each unit."""

    datetime = dict()
    for unit in units:
        nano_unit = time_unit_to_nanoseconds(unit)
        nano_sec =  dataset[unit]* (2 * np.pi / nano_unit)
        datetime[unit + '_sin'] = np.sin(nano_sec)
        datetime[unit + '_cos'] = np.cos(nano_sec)
    return pd.DataFrame(datetime, dtype=np.float32)

In [ ]:
dateencoded=datetime_encoded(raw_data, ['day'])
dateencoded

In [ ]:
route = '005'
direction = 'i'
route_start = 0
route_len =32
missing_rate = 0.3
missing_type = ['random','linear','block','mixed']
missing_type_ind = 2
num_masks = 10
data_type=['source_train','source_test','target_train','target_test']
data_type_ind = 0

if direction=='i':
    route1=node.query('route==@route and direction==@direction').sort_values('mile_post',ascending=False)['milepost']
else:
    route1=node.query('route==@route and direction==@direction').sort_values('mile_post',ascending=True)['milepost']
hot_map_data = raw_data.loc[:][route1]


hot_map_data=hot_map_data.values
hot_map_data=hot_map_data.reshape(-1,288,hot_map_data.shape[-1])
hot_map_data.shape

np.random.shuffle(hot_map_data)

train_ratio = 0.8
test_ratio = 0.2
source_train_data = hot_map_data[:int(hot_map_data.shape[0]*train_ratio)]
test_data = hot_map_data[int(hot_map_data.shape[0]*train_ratio):int(hot_map_data.shape[0]*(train_ratio+test_ratio))]
target_train_data = hot_map_data[int(hot_map_data.shape[0]*(train_ratio+test_ratio)):]
print(source_train_data.shape,test_data.shape,target_train_data.shape)


In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(20, 10))
hot_map = ax.imshow(hot_map_data[6,:,:].T, cmap='jet', interpolation='nearest')
plt.colorbar(hot_map)
plt.show()

In [22]:
test_dir=r'D:\WorkPath\Models\ImputeFormer\Data\source_test_Seattle'
train_dir=r'D:\WorkPath\Models\ImputeFormer\Data\source_train_Seattle'
# os.makedirs(train_dir)
# os.makedirs(test_dir)

In [25]:
train_ratio = 0.8
test_ratio = 0.2
missing_rate = 0.7

random.seed(0)
x = list(range( hot_map_data.shape[0] ))
random.shuffle(x)

# for missing_type_ind in range(4):
for missing_type_ind in [3]:
    for route_start in range(0,64-32,6):
        for time_start in [7,11,15,19]:
            
            save_path = train_dir
            save_path_test = test_dir

            

            # dataset = MultiMaskTimeSeriesDataset(hot_map_data[ 5:7,
            #                                     time_start*12:(time_start+4)*12,route_start:route_start+route_len],
            #                                      dateencoded.values[time_start*12:(time_start+4)*12,:],
            #                                     missing_rate, missing_type[missing_type_ind], num_masks)
            # dataset_file = os.path.join(r'D:\WorkPath\Models\Plot\Seattle405', 
            #                             'Seattle_405_speed_{}-{}_{}_{}_{}.pkl'.format
            #                             (route, time_start ,route_start ,int(missing_rate*100) , missing_type[missing_type_ind]))
            # pd.to_pickle(dataset, dataset_file)
            

            # dataset = MultiMaskTimeSeriesDataset(hot_map_data[ x[0:int(train_ratio* hot_map_data.shape[0]) ],
            #                                     time_start*12:(time_start+4)*12,route_start:route_start+route_len],
            #                                      dateencoded.values[time_start*12:(time_start+4)*12,:],
            #                                     missing_rate, missing_type[missing_type_ind], num_masks)

            # dataset_file = os.path.join(save_path, 
            #                             'Seattle_405_speed_{}{}_{}_{}_{}.pkl'.format
            #                             (route, time_start ,route_start ,int(missing_rate*100) , missing_type[missing_type_ind]))
            # pd.to_pickle(dataset, dataset_file)
            
            dataset = MultiMaskTimeSeriesDataset(hot_map_data[ x[int(train_ratio* hot_map_data.shape[0]): ],
                                                time_start*12:(time_start+4)*12,route_start:route_start+route_len],
                                                 dateencoded.values[time_start*12:(time_start+4)*12,:],
                                                missing_rate, missing_type[missing_type_ind], 5)
            # 保存数据集
            dataset_file = os.path.join(save_path_test, 
                                        'Seattle_2015_speed_{}{}_{}_{}_{}.pkl'.format
                                        (route, time_start ,route_start ,int(missing_rate*100) , missing_type[missing_type_ind]))
            pd.to_pickle(dataset, dataset_file)